In [2]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import imp
import help_tools as ht
HTML("<style>.container { width: 120% !important; }</style>")

In [1]:
api_key_tmdb = '' 
api_key_omdb = ''

In [8]:
# откуда накидывать id людей
# https://www.themoviedb.org/person/7030-henry-goodman

actor_dict = {
            'joaquin-phoenix' : 73421,
            'colin-farrell' : 72466,
            'leonardo-dicaprio' : 6193,
            'javier-bardem' : 3810,
            'anthony-hopkins' : 4173,
            'brad-pitt' : 287,
            'antonio-banderas': 3131,
            'matthew-mcconaughey' : 10297,
            'steve-buscemi' : 884,
            'tim-roth' : 3129,
            'mads-mikkelsen' : 1019,
            'scarlett-johansson' : 1245,
            'til-schweiger' : 1844,
            'george-clooney' : 1461,
            'tom-hanks' : 31,
            'robert-de-niro' : 380,
            'adrien-brody' : 3490,
            'matt-damon' : 1892,
            'edward-norton' : 819,
            'robert-pattinson' : 11288,
            'jude-law' : 9642,
            'cillian-murphy' : 2037,
            'benicio-del-toro' : 1121,
            'timothee-chalamet' : 1190668
             }

director_dict = {
#                 'quentin-tarantino' : 138,
#                 'david-fincher' : 7467,
#                 'jim-jarmusch' : 4429,
#                 'guy-ritchie' : 956,
#                 'christopher-nolan' : 525,
#                 'darren-aronofsky' : 6431,
#                 'martin-mcdonagh' : 54472,
#                 'pedro-almodovar' : 309,
#                 'paolo-sorrentino' : 56194,
#                 'cristian-mungiu' : 20657,
#                 'yorgos-lanthimos' : 122423,
#                 'bong-joon-ho' : 21684,
#                 'lars-von-trier' : 42,
#                 'woody-allen' : 1243
}

# фильмы которые не хочется чекать (уже просмотренные или не интересные)
excl_films = [335977, 799583, 882569, 800158, 872585, 696506, 1029955, 693134, 549509, 956842, 877817]

# предварительные фильтры (по имеющимся параметрам в фильмографии - чтобы дальше не тратить запросы на детализацию)
vote_average_min = 5
vote_count_min = 100
release_date_min = '2024-01-01'
language = 'ru' # добавлять русскояз названия если есть в прокате 

In [9]:
# ПОЛУЧАЕМ ИНФУ ИЗ TMDB + OMDB (РАБОТА С API)
flt_dict = {'vote_average_min' : vote_average_min, 
            'vote_count_min' : vote_count_min,
            'release_date_min' : release_date_min,
            'language' : language
           }

# получаем инфу по списку актеров
df1 = ht.get_actors_filmo(api_key_tmdb, actor_dict, excl_films = excl_films, flt_dict = flt_dict)
# получаем инфу по списку режиссеров
df2 = ht.get_directors_filmo(api_key_tmdb, director_dict, excl_films = np.append(df1.movie_id.values, excl_films), flt_dict = flt_dict)
# итого
df = pd.concat([df1, df2], ignore_index = True)
df_final = ht.get_imdb_info(api_key_omdb, df)

In [14]:
# ПОСТ ФИЛЬТРАЦИЯ И СОРТИРОВКИ ИЗ ДОП УСЛОВИЙ (НЕТ РАБОТЫ С API)
post_excl_films = []
ht.get_filter(df_final, 
           vote_count_min=100, # vote cnt on tmdb
           runtime_min = 70, # film duration
           vote_average_min = 5, # rating по tmdb
           imdb_rating_min = 6, 
           release_date_min = '2023-01-01',
           genres_flt = ['Animation', 'Documentary'], # жанры которые неинтересно отображать
           excl_films = np.append(excl_films, post_excl_films), # фильмы которые неинтересно отображать
#            drop_columns=['movie_status', 'movie_id', 'countries'] # какие колонки скрыть
           drop_columns=['movie_status', 'countries'] # какие колонки скрыть
          ).sort_values(by='imdb_rating', ascending=False)

,person,person_type,title,original_title,directors,release_date,imdb_rating,awards,genres,runtime,imdb_id,vote_average,vote_count,movie_id
7,edward-norton,actor,Боб Дилан: Никому Не Известный,A Complete Unknown,James Mangold,2024-12-18,7.7,Nominated for 8 Oscars. 24 wins & 127 nominati...,"['Drama', 'Music']",140,tt11563598,7.159,961,661539
1,matthew-mcconaughey,actor,Дэдпул и Росомаха,Deadpool & Wolverine,Shawn Levy,2024-07-24,7.6,24 wins & 63 nominations total,"['Action', 'Comedy', 'Science Fiction']",128,tt6263850,7.600,7192,533535
4,jude-law,actor,Безмолвное братство,The Order,Justin Kurzel,2024-12-05,6.8,2 wins & 6 nominations total,"['Crime', 'Drama', 'Thriller']",116,tt26625693,6.575,603,1082195
8,cillian-murphy,actor,Мелочи жизни,Small Things Like These,Tim Mielants,2024-11-01,6.8,4 wins & 11 nominations total,['Drama'],99,tt27196021,6.900,255,1102493
9,antonio-banderas,actor,Приключения Паддингтона 3,Paddington in Peru,Dougal Wilson,2024-11-08,6.7,2 wins & 5 nominations total,"['Family', 'Comedy', 'Adventure']",106,tt5822536,6.851,372,516729
3,matt-damon,actor,Воображаемые Друзья,IF,John Krasinski,2024-05-08,6.4,7 nominations total,"['Comedy', 'Fantasy', 'Family']",104,tt11152168,7.068,1347,639720
6,george-clooney,actor,Воображаемые Друзья,IF,John Krasinski,2024-05-08,6.4,7 nominations total,"['Comedy', 'Fantasy', 'Family']",104,tt11152168,7.068,1347,639720
0,tom-hanks,actor,Тогда. Сейчас. Потом,Here,Robert Zemeckis,2024-10-30,6.3,2 wins & 5 nominations total,['Drama'],105,tt18272208,6.678,452,940139
5,jude-law,actor,Игра королевы,Firebrand,Karim Aïnouz,2024-03-27,6.3,1 win & 2 nominations total,"['Drama', 'History', 'Thriller']",120,tt15000156,6.365,104,848439
2,matt-damon,actor,Зачинщики,The Instigators,Doug Liman,2024-08-02,6.2,1 nomination total,"['Crime', 'Comedy', 'Action']",102,tt24169886,6.200,474,1059064


### Spotify

In [29]:
# get id + secret on your : https://developer.spotify.com/dashboard/e7bd2deea29b4fe1a33392499c720f45
client_id = ''
client_secret = ''

In [15]:
# посмотреть айдишки наиболее популярных исполнителей по имени
artist_name = 'massive'
top_cnt = 5
ht.get_top_n_id_by_name(client_id, client_secret, artist_name, top_cnt)

['Massive Attack - 6FXMGgJwohJLUSr5nVlf9X - popularity: 71',
 'Hang Massive - 6bkF6GDcyXZn2T0D5Fwldl - popularity: 48',
 'Scratch Massive - 5udXThiZTVoHa4g0GDjgxA - popularity: 42',
 'Massivebass - 5p9GQuGCJcfovjMcDk3ZyI - popularity: 38',
 'Massive B - 36fJ2Mx3ktclhSlBbsUbFY - popularity: 41']

In [16]:
artist_list = [
#                 '132sZpCaM8ie6byAEcOcRs', # Haelos
               '2VYQTNDsvvKN9wmU5W7xpj', # manson
                '7Ln80lUS6He07XvHI8qqHH', # arctic monk
                '2NPduAUeLVsfIauhRwuft1', # nightwish
#                 '5nPOO9iTcrs9k6yFffPxjH', # Röyksopp
#                 '2a5G7JLmVJNjfFNg8rwLcP', # Till Lindemann
              '3w4VAlllkAWI6m0AV0Gn6a', # hurts,
               '4njdEjTnLfcGImKZu1iSrz', # awolnation
               '3Bd1cgCjtCI32PYvDC3ynO', # london gram
               '3XHO7cRUPCLOr6jwp8vsx5', # alt j
               '163tK9Wjr9P9DmM0AVK7lm', # lorde
               '3iOvXCl6edW5Um0fXEBRXy', # xx
              '0RqtSIYZmd4fiBKVFqyIqD', # 30 second
               '5Pwc4xIPtQLFEnJriah9YJ', # one republ
              '1WgXqy2Dd70QQOU7Ay074N', # aurora,
                '4Z8W4fKeB5YxbusRsdQVPb', # radiohead
               '0BEI7i5sgUuivcfwXLzFmM', # serj tank
               '3wRt3iJpZDOg73CTUkfv5C', # svrcina
#                 '0YCqg8X5gx0MremW64Ud1h', # ддт
                '6mdiAmATAx73kdxrNrnlao', # iron m
                '2ye2Wgw4gimLv2eAKyk1NB', # metallica
                '1moxjboGR7GNWYIMWsRjgG', # florence + m
                '1AZ30JnvQU1pbX6sbRE0Yn', # poets of the fall
                '2N0vFuOoMtAQfBmhsRo24e', # kalanda
                '4Z8W4fKeB5YxbusRsdQVPb', # radiohead
                '1Ffb6ejR6Fe5IamqA5oRUF', # bring me the horizon
                '00FQb4jTyendYWaN8pK0wa', # lana del ray
                '49qiE8dj4JuNdpYGRPdKbF', # stone sour
                '6wWVKhxIU2cEi0K81v7HvP', # rammstein
                '5nGIFgo0shDenQYSE0Sn7c', # Evanescence
                '6XyY86QOPPrYVGvF9ch6wz', # linkin park
                '53KwLdlmrlCelAZMaLVZqU', # james blake
                '6FXMGgJwohJLUSr5nVlf9X' # massive attack
]

artist_list = list(np.unique(artist_list))

df_spotify = ht.sp_get_albums_info(client_id, client_secret, artist_list)

In [21]:
excl_album_list = ['The Greatest Love', 'From Zero', 'One Assassination Under God - Chapter 1', 
                    'What Happened To The Heart?', 
                   'From Zero: A Cappellas', 'Papercuts: Instrumentals', 'The Phantom Five', '1200 Beats Per Minute']

In [22]:
# ПОСТ ФИЛЬТРАЦИЯ (АПИ НЕ РАБОТАЕТ ЗДЕСЬ)
res = ht.sp_get_filter(df_spotify, 
                release_date_min = '2024-01-01', 
                release_date_max = '2030-01-01',
#                 album_flt = ['deluxe', 'live', 'edition', 'soundtrack', 'remix'], # исключаем слова в названии альбомов
                album_flt = [  'edition', 'soundtrack', 'remix', 'deluxe'], # исключаем слова в названии альбомов
                album_type_flt = ['single'], # убрать эти типы
                max_artists_in_album = 1,
                other_album_author = False, # false когда не хотим чтобы это был фит артиста в альбом другого чувака
#                  columns_excl = ['uri', 'album_type'] # какие колонки не отображать здесь
                 columns_excl = [] # какие колонки не отображать здесь
                 )

res[~res.album.isin(excl_album_list)]

,artist,album,release_date,tracks_cnt,uri,album_artists,album_type
1,Nightwish,Yesterwynde,2024-09-20,23,spotify:album:39CVUoSmw8915LRClkNH0o,['Nightwish'],album
5,Linkin Park,From Zero: A Cappellas + Instrumentals,2024-11-14,21,spotify:album:7qKEQkRXySP0rhjtXJksfg,['Linkin Park'],album
8,Linkin Park,Papercuts,2024-04-12,20,spotify:album:1X472EvsSqH09RyrqbtyXU,['Linkin Park'],album
9,Kalandra,A Frame of Mind,2024-09-13,10,spotify:album:7tmd1LZ4ZT9eoA4Jl1MNFy,['Kalandra'],album
10,James Blake,Playing Robots Into Heaven (Endel Focus Sounds...,2024-02-02,12,spotify:album:1yomJcRq9gy87Z6VwbWrGH,['James Blake'],album
11,Bring Me The Horizon,POST HUMAN: NeX GEn,2024-05-24,16,spotify:album:1k7OXnGQPV4zF3seDwRroD,['Bring Me The Horizon'],album
